In [1]:
import torch, argparse, pandas, numpy

from sklearn.metrics import accuracy_score, mean_squared_error


from hahadataset import HahaDataset

from transformers import GPT2ForSequenceClassification, GPT2Tokenizer
from transformers import TrainingArguments, Trainer

In [2]:
def metrics_acc(eval_pred):

    labels = eval_pred.label_ids
    preds = eval_pred.predictions.argmax(-1)
    acc = accuracy_score(labels, preds)

    return {"accuracy": acc}

def metrics_rmse(eval_pred):
    labels = eval_pred.label_ids
    preds = eval_pred.predictions
    rmse = mean_squared_error(labels,preds)

    rmse = numpy.float64(rmse)

    return{"rmse": rmse}

In [3]:
if __name__ == '__main__':

    parser = argparse.ArgumentParser()

    parser.add_argument('--load_from_checkpoint', type=str)
    parser.add_argument('--continue_training', type=str)
    parser.add_argument('--output_directory', type=str,default="output_dir")
    parser.add_argument('--tokenizer_path', type=str)
    parser.add_argument('--max_len', type=int, default=256)
    parser.add_argument('--max_steps', type=int, default=500)
    parser.add_argument('--batch_size', type=int, default=8)
    parser.add_argument('--num_gpus', type=int, default=4)
    parser.add_argument("-f", "--fff", help="a dummy argument to fool ipython", default="1")
    args = parser.parse_args()
    task = 'offense_rating'

    tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
    tokenizer.padding_side = "left"
    tokenizer.pad_token = tokenizer.eos_token


    train_dataset = HahaDataset(input_file='./../Data/train_data/train.csv', tokenizer=tokenizer, max_len=256, task=task,
                                split='train')
    eval_dataset = HahaDataset(input_file='./../Data/train_data/train.csv', tokenizer=tokenizer, max_len=256, task=task,
                               split='eval')
    test_dataset = HahaDataset(input_file='./../Data/test_data/public_test.csv', tokenizer=tokenizer, max_len=256, task=task,
                               split='test')

    model = GPT2ForSequenceClassification.from_pretrained('gpt2', num_labels=1)
    
    model.resize_token_embeddings(len(tokenizer))

    # fix model padding token id
    model.config.pad_token_id = model.config.eos_token_id


    warmup_steps = int(args.max_steps * .01)

    training_args = TrainingArguments(
        output_dir=args.output_directory,
        max_steps=args.max_steps,
        per_device_train_batch_size=args.batch_size,
        logging_steps=25,
        save_total_limit=1,
#         evaluate_during_training=True,
        eval_steps=50,
        learning_rate=2e-5,
        warmup_steps=warmup_steps,
        load_best_model_at_end=True,
        metric_for_best_model='eval_rmse',
        disable_tqdm=True
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        compute_metrics=metrics_rmse,
    )

    trainer.train()

    predictions = trainer.predict(test_dataset)
    humor_rating_preds = [pred[0] for pred in predictions.predictions]

    output_list = []
    for pred in humor_rating_preds:
        temp = {}
        temp['offense_rating'] = pred
        output_list.append(temp)

    out_df = pandas.DataFrame(output_list)
    out_df.to_csv('submission_offense_rating.csv', index_label='id')


Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


{'loss': 5.269570922851562, 'learning_rate': 1.9191919191919194e-05, 'epoch': 0.03125}
{'loss': 1.1091950225830078, 'learning_rate': 1.8181818181818182e-05, 'epoch': 0.0625}
{'loss': 0.8685845184326172, 'learning_rate': 1.7171717171717173e-05, 'epoch': 0.09375}
{'loss': 0.8746347045898437, 'learning_rate': 1.616161616161616e-05, 'epoch': 0.125}
{'loss': 1.3312753295898438, 'learning_rate': 1.5151515151515153e-05, 'epoch': 0.15625}
{'loss': 0.8697610473632813, 'learning_rate': 1.4141414141414143e-05, 'epoch': 0.1875}
{'loss': 0.938218994140625, 'learning_rate': 1.3131313131313132e-05, 'epoch': 0.21875}
{'loss': 0.9558061981201171, 'learning_rate': 1.2121212121212122e-05, 'epoch': 0.25}
{'loss': 1.0509859466552733, 'learning_rate': 1.1111111111111113e-05, 'epoch': 0.28125}
{'loss': 1.0724846649169921, 'learning_rate': 1.0101010101010103e-05, 'epoch': 0.3125}
{'loss': 0.8953407287597657, 'learning_rate': 9.090909090909091e-06, 'epoch': 0.34375}
{'loss': 0.7770709228515625, 'learning_rate'